In [ ]:
import pandas as pd
import os, time, whois
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# --- Cấu hình ---
INPUT_FILE = "dataset.csv"  # File gốc
OUTPUT_FILE = "f25-whois-age.csv"  # File lưu tiến độ
BATCH_SIZE = 10000  # Số domain / batch
MAX_THREADS = 300  # Số luồng song song
CACHE_FILE = "whois_cache.csv"  # File cache WHOIS

# --- Đọc dữ liệu ---
try:
    data = pd.read_csv(INPUT_FILE)
except FileNotFoundError:
    print(f"Không tìm thấy file: {INPUT_FILE}. Kiểm tra lại đường dẫn!")
    exit()

if 'domain' not in data.columns:
    print("Không tìm thấy cột 'domain' trong file CSV!")
    exit()

# --- Kiểm tra tiến độ ---
if os.path.exists(OUTPUT_FILE):
    processed_data = pd.read_csv(OUTPUT_FILE)
    print(f"Tiếp tục từ file: {OUTPUT_FILE}.")
    data = processed_data  # Dùng dữ liệu đã có
else:
    print("Bắt đầu mới! Chưa có file tiến độ.")
    data['f25'] = None  # Thêm cột f25 nếu chưa có

processed_count = data['f25'].notna().sum()
print(f"Đã xử lý: {processed_count}/{len(data)} domain.")

# --- Đọc cache WHOIS (nếu có) ---
whois_cache = {}
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE)
    whois_cache = dict(zip(cache_df['domain'], cache_df['f25']))
    print(f"Đã load {len(whois_cache)} domain từ cache.")

# --- Hàm lấy WHOIS Age ---
def get_whois_age(domain):
    if domain in whois_cache:
        return domain, whois_cache[domain]  # Trả về từ cache nếu có

    try:
        w = whois.whois(domain)
        if w.creation_date:
            age = (pd.Timestamp.now() - pd.to_datetime(w.creation_date[0])).days
            whois_cache[domain] = age  # Lưu vào cache
            return domain, age
    except:
        return domain, None
    return domain, None

# --- Xử lý theo batch ---
for i in range(0, len(data), BATCH_SIZE):
    batch = data.iloc[i:i + BATCH_SIZE]
    batch = batch[batch['f25'].isna()]  # Bỏ qua domain đã có WHOIS

    if batch.empty:
        continue  # Skip batch đã hoàn thành

    print(f"🔹 Xử lý batch {i//BATCH_SIZE} ({len(batch)} domain)...")
    start = time.time()

    results = {}
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = {executor.submit(get_whois_age, d): d for d in batch['domain']}
        for f in tqdm(as_completed(futures), total=len(futures), desc=f"Batch {i//BATCH_SIZE}"):
            domain, age = f.result()
            results[domain] = age

    # Cập nhật vào DataFrame
    for domain, age in results.items():
        data.loc[data['domain'] == domain, 'f25'] = age

    # Lưu tiến độ sau mỗi batch
    data.to_csv(OUTPUT_FILE, index=False)

    # Lưu cache WHOIS
    cache_df = pd.DataFrame(list(whois_cache.items()), columns=['domain', 'f25'])
    cache_df.to_csv(CACHE_FILE, index=False)

    end = time.time()
    print(f"Batch {i//BATCH_SIZE} xong! ({round(end-start,2)}s)")

print(f"\n Hoàn thành! Kết quả lưu tại: {OUTPUT_FILE}")


Không tìm thấy file: dataset.csv. Kiểm tra lại đường dẫn!


NameError: name 'data' is not defined

: 